In [162]:
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

In [163]:
df = pd.read_csv('data.csv')

df.head()

,brand,price,year,mileage,damage_type,number_of_owners,number_of_services
0,Dodge,14499,2015,115373,unknown,2,9
1,Kia,9995,2017,84579,none,1,3
2,Chevrolet,11995,2017,77315,none,3,13
3,Nissan,9995,2015,94783,none,1,15
4,Volkswagen,11977,2021,137978,none,1,13


In [164]:
damage_tokenizer = LabelEncoder()

In [165]:
df['tokenized_damage'] = damage_tokenizer.fit_transform(df['damage_type'])

df.head()

,brand,price,year,mileage,damage_type,number_of_owners,number_of_services,tokenized_damage
0,Dodge,14499,2015,115373,unknown,2,9,3
1,Kia,9995,2017,84579,none,1,3,2
2,Chevrolet,11995,2017,77315,none,3,13,2
3,Nissan,9995,2015,94783,none,1,15,2
4,Volkswagen,11977,2021,137978,none,1,13,2


In [166]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.hidden1 = nn.Linear(10, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.hidden1(x))
        #x = F.leaky_relu(self.hidden2(x))
        #x = F.leaky_relu(self.hidden3(x))
        x = self.output(x)
        return x

toyota_model = Net()
honda_model = Net()

In [167]:
toyota_optim = optim.Adam(toyota_model.parameters(), lr=0.01)
honda_optim = optim.Adam(honda_model.parameters(), lr=0.01)
loss = nn.MSELoss()

In [186]:
epochs = 50000

In [187]:
for epoch in range(epochs):
    for index, row in df.iterrows():

        if row['brand'] != 'Toyota' and row['brand'] != 'Honda':
            continue

        if row['brand'] == 'Toyota':
            toyota_optim.zero_grad()
            inputs = torch.from_numpy(np.array([
                float(row['year']),  # Convert to float
                float(row['mileage']),  # Convert to float
                float(row['tokenized_damage']),  # Convert to float
                float(row['number_of_owners']),  # Convert to float
                float(row['number_of_services']),  # Convert to float
            ])).float().requires_grad_(True).unsqueeze(0)
            target = torch.tensor([float(row['price'])]).float().unsqueeze(0)
            target = target.repeat(1, 5)  # Repeat the target tensor to match the size of the inputs tensor
            loss_value = loss(inputs, target)  # Calculate loss
            loss_value.backward()  # Backpropagation
            toyota_optim.step()  # Update Toyota weights
        
        elif row['brand'] == 'Honda':
            honda_optim.zero_grad()
            inputs = torch.from_numpy(np.array([
                float(row['year']),  # Convert to float
                float(row['mileage']),  # Convert to float
                float(row['tokenized_damage']),  # Convert to float
                float(row['number_of_owners']),  # Convert to float
                float(row['number_of_services']),  # Convert to float
            ])).float().requires_grad_(True).unsqueeze(0)
            target = torch.tensor([float(row['price'])], requires_grad=False).float().unsqueeze(0)
            target = target.repeat(1, 5) # Repeat the target tensor to match the size of the inputs tensor
            # Ensure loss is a function:
            loss = nn.MSELoss()  # Example, replace with your actual loss function

            loss_value = loss(inputs, target)  # Now you can call the loss function
            loss_value.backward()
            honda_optim.step()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}/{epochs}, {epoch/epochs*100}% done')



Epoch 0/50000, 0.0% done
Epoch 1000/50000, 2.0% done
Epoch 2000/50000, 4.0% done
Epoch 3000/50000, 6.0% done
Epoch 4000/50000, 8.0% done
Epoch 5000/50000, 10.0% done
Epoch 6000/50000, 12.0% done
Epoch 7000/50000, 14.000000000000002% done
Epoch 8000/50000, 16.0% done
Epoch 9000/50000, 18.0% done
Epoch 10000/50000, 20.0% done
Epoch 11000/50000, 22.0% done
Epoch 12000/50000, 24.0% done
Epoch 13000/50000, 26.0% done
Epoch 14000/50000, 28.000000000000004% done
Epoch 15000/50000, 30.0% done
Epoch 16000/50000, 32.0% done
Epoch 17000/50000, 34.0% done
Epoch 18000/50000, 36.0% done
Epoch 19000/50000, 38.0% done
Epoch 20000/50000, 40.0% done
Epoch 21000/50000, 42.0% done
Epoch 22000/50000, 44.0% done
Epoch 23000/50000, 46.0% done
Epoch 24000/50000, 48.0% done
Epoch 25000/50000, 50.0% done
Epoch 26000/50000, 52.0% done
Epoch 27000/50000, 54.0% done
Epoch 28000/50000, 56.00000000000001% done
Epoch 29000/50000, 57.99999999999999% done
Epoch 30000/50000, 60.0% done
Epoch 31000/50000, 62.0% done
Epoc

</h>BELIEVE IT!<h>

In [191]:
# HOW COME THIS SHIT IS FUCKING BROKEN THAN A MOTHERFUCKER BIIIIIIIIIIIIIIIIIIIIIIIIIITCH?

print(f"${toyota_model(torch.tensor([2019, 50000, 2, 2, 13]).float()).item()}")
print(f"${honda_model(torch.tensor([2019, 50000, 2, 2, 13]).float()).item()}")

$795.2431640625
$-736.2704467773438


In [192]:
df.sample(5)

,brand,price,year,mileage,damage_type,number_of_owners,number_of_services,tokenized_damage
15,Honda,20933,2021,62620,none,1,3,2
16,Honda,18533,2020,42289,none,2,7,2
1,Kia,9995,2017,84579,none,1,3,2
12,Honda,16495,2015,37334,unknown,3,9,3
19,Mazda,23250,2022,29374,none,1,4,2


In [198]:
honda_model(torch.tensor([0, 0, 2, 2, 13]).float()).item()

0.42109060287475586